# Code Profiling : Analyze the cProfile output

We will use the `pstats` module to read and analyze the outupt of the profiler.


In [1]:
import pstats
mystats = pstats.Stats("profiler_output.txt")

## Display the top 15 functions that took the longest time to run

Even in such a short script, python called millions of function! A lot of them are super quick and really not what we are after. Typically, we want to find the few functions that are bottlenecks in our code.

The function below sorts the functions by which took the longest to run, and prints out only the stats of the top 15 functions that took longest to run. The columns below mean:
  * ncalls: number of times this function was called
  * tottime: the total execution time spent in this code NOT including calls to other functions
  * percall: this first percall divides tottime by ncalls. The amount of time per call spent solely in this function.
  * cumtime: the total execution time spent in this code INCLUDING calls to sub functions. `cumtime > tottime always.
  * percall: second percall divides cumtime by ncalls
  * filename: the name of the function being considered

Both `tottime` and `cumtime` are useful. A function with a high `tottime` means we should focus on speeding up this function. A function with only a high `cumtime` means we should see what this function is calling to improve runtime.
  * `_newton_solver()` is a function with a higher tottime. We should look at the lines in that function to check out how to speed it up.
  * `_logl()` is a function with a low tottime but a very high `cumtime`. We should look at the functions it calls to figure out what takes up so much time.

Generally, we want to put some filters on `print_stats`, becuase otherwise there will be so much printed out it is unmanageable. 


In [15]:
mystats.sort_stats("time").print_stats(25) # note that you need to display the top 25 to see _logl

Wed Jun 24 09:43:50 2020    profiler_output.txt

         2984019 function calls (2943240 primitive calls) in 24.113 seconds

   Ordered by: internal time
   List reduced from 4773 to 25 due to restriction <25>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      732    6.243    0.009    6.833    0.009 /Users/bluez3303/Documents/GitHub/orbitize/orbitize/kepler.py:173(_newton_solver)
      732    5.898    0.008   16.214    0.022 /Users/bluez3303/Documents/GitHub/orbitize/orbitize/kepler.py:17(calc_orbit)
      366    1.707    0.005    1.732    0.005 /Users/bluez3303/Documents/GitHub/orbitize/orbitize/lnlike.py:8(chi2_lnlike)
      732    1.021    0.001    8.776    0.012 /Users/bluez3303/Documents/GitHub/orbitize/orbitize/kepler.py:111(_calc_ecc_anom)
23022/22015    0.856    0.000    1.233    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      732    0.768    0.001    0.768    0.001 /Users/bluez3303/Documents/GitHub/orbitize/orbit

We can also apply multiple filters. For example let's look at the top 15 numpy functions that took the most time to run. Note that the order of the filtering matters. This command first selects every function with numpy in the name, followed by taking the top 15. Calling `print_states(15, 'numpy')` would pick the top 15 longest runtime functions, and downselecting which has numpy in the name from those. That would give us less than 15 numpy functions.

In [10]:
mystats.sort_stats("time").print_stats('numpy', 15)

Wed Jun 24 09:43:50 2020    profiler_output.txt

         2984019 function calls (2943240 primitive calls) in 24.113 seconds

   Ordered by: internal time
   List reduced from 4773 to 492 due to restriction <'numpy'>
   List reduced from 492 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
23022/22015    0.856    0.000    1.233    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    29538    0.441    0.000    0.441    0.000 {built-in method numpy.array}
     2196    0.403    0.000    0.403    0.000 {method 'normal' of 'numpy.random.mtrand.RandomState' objects}
     2196    0.231    0.000    0.231    0.000 {method 'uniform' of 'numpy.random.mtrand.RandomState' objects}
      366    0.090    0.000    0.090    0.000 {method 'repeat' of 'numpy.ndarray' objects}
44190/44188    0.044    0.000    0.055    0.000 {method 'view' of 'numpy.ndarray' objects}
      366    0.040    0.000    0.094    0.000 /Users/bluez

## Activity Part 1: Analyze the output

Generate the output above on your own by running `python -m cProfile -o profiler_output.txt profile_orbitize.py` and answer the following questions by analyzing the output.

1. Which function takes up the most runtime (not including calls to sub-functions)?

2. Which function takes up the most runtime (including calls to sub-functions)?

3. Which function is called the most? Which `orbitize` function is called the most?

4. If we had the magical ability to speed up one function by a factor of 2, which function should we speed up? What is the improvement in end-to-end runtime of the script?



In [ ]:
# 1: `_newton_solver` takes up the most runtime because it has the highest tottime
# 2: [[ mystats.sort_stats("cumtime").print_stats() ]] `run_sampler` takes up the most runtime including calls to sub-functions because 
#    it has the highest cumtime.
# 3: [[ mystats.sort_stats("ncalls").print_stats() ]] The function called the most is the built-in method `isinstance`. The orbitize function 
#    called most is `draw_samples`. 
# 4: We should choose to speed up `_newton_sampler`. On my laptop, `_newton_sampler` had a tottime of 6.243 s, and the entire script took 
#    24.113 s. Speeding up `_newton_sampler` by a factor of two would decrease its tottime to 3.122 s, and so decrease the end-to-end runtime 
#    of the script to 24.113 - 3.122 = 21.011 s.

## Activity Part 2: Investigate why `_logl` takes so long

`_logl()` is a helper function in `orbitize!` to compute the log likelihood of the data given the model. The `_logl()` function itself has a short runtime but it is calling something that takes a long time that makes it have a long `cumtime`. We can use the `print_callees()` function to look at the stats of all the functions it calls.

We can see that `compute_model` is the function with the highest cumtime, but its tottime is low, so we something it calls takes all the time. We must dig deeper! Keep digging down recursively to find what function called within `_logl()` that takes the longest.

In [20]:
mystats.print_callees('_logl')
mystats.print_callees('compute_model')
mystats.print_callees('calc_orbit')
mystats.print_callees('_calc_ecc_anom')

# Aha! `_newton_solver` appears to be the culprit, in line with the previous activity's findings. Pesky orbits. 

Ordered by: internal time
   List reduced from 4773 to 1 due to restriction <'_logl'>

Function                                                                  called...
                                                                              ncalls  tottime  cumtime
/Users/bluez3303/Documents/GitHub/orbitize/orbitize/sampler.py:46(_logl)  ->     366    1.707    1.732  /Users/bluez3303/Documents/GitHub/orbitize/orbitize/lnlike.py:8(chi2_lnlike)
                                                                                 366    0.189   13.721  /Users/bluez3303/Documents/GitHub/orbitize/orbitize/system.py:184(compute_model)
                                                                                1464    0.002    0.004  /Users/bluez3303/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/table.py:1600(__getitem__)
                                                                                 366    0.001    0.130  <__array_function__ internals>:2(nansum)

## Activity Part 3: Which function calls `numpy.array` the most?

`numpy.array` is a popular function because it gets called anytime a new numpy array gets created. We can use `print_callers()` to see which functions call it to look into potentially reducing the number of array creations to speed up the code. Which function in `orbitize` calls `numpy.array` the most times per function call? (it's harder than it looks)

In [33]:
mystats.print_callers('numpy.array')

# The orbitize functions that call `numpy.array` are `calc_orbit`, `_newton_solver`, `_logl`, `sampler.__init__`, `reject`, and `run_sampler`.
#   To find which one calls `numpy.array` the most times *per function call*, divide ncalls in the output below by the number of times the function
#   itself is called. 

mystats.sort_stats("ncalls").print_stats('orbitize')

# `calc_orbit` : (total number of times it calls `numpy.array`: 366) / (total number of times it is called : 732) = 0.5
# `_newton_solver` : 1464 / 732 = 2
# `_logl` : 732 / 366 = 0.5
# `sampler.__init__` : 3 / 1 = 3
# `reject` : 1098 / 366 = 3
# `run_sampler` : 1 / 1 = 1

# Therefore, `sampler.__init__` calls `numpy.array` the most *per time it is called*. In other words, the code for `sampler.__init__`
#   contains the most lines that call `numpy.array`.

Ordered by: call count
   List reduced from 4773 to 1 due to restriction <'numpy.array'>

Function                       was called by...
                                   ncalls  tottime  cumtime
{built-in method numpy.array}  <-     366    0.001    0.001  /Users/bluez3303/Documents/GitHub/orbitize/orbitize/kepler.py:17(calc_orbit)
                                     1464    0.210    0.210  /Users/bluez3303/Documents/GitHub/orbitize/orbitize/kepler.py:173(_newton_solver)
                                      732    0.004    0.004  /Users/bluez3303/Documents/GitHub/orbitize/orbitize/sampler.py:46(_logl)
                                        3    0.000    0.000  /Users/bluez3303/Documents/GitHub/orbitize/orbitize/sampler.py:104(__init__)
                                     1098    0.006    0.006  /Users/bluez3303/Documents/GitHub/orbitize/orbitize/sampler.py:274(reject)
                                        1    0.000    0.000  /Users/bluez3303/Documents/GitHub/orbitize/orbitize/

# Code Profiling Strategy

Generally, it is important to focus your effort on speeding up the 1 or 2 slowest functions. If you speed up something that takes 50% of the runtime by a factor of 100, you speed up the code by a factor of 2. Even if a piece of code takes 10% of the runtime, a factor of 2 improvement (which by itself is often hard to get) only gives you a 5% improvement in the end. **Focus your effort on optimizing the runtime of the one or two functions that take the most amount of time.**

For all else, it is generally more important to optimize code readibility over runtime. Code that is more readible is less prone to bugs and eaiser to maintain. If a piece of code takes only 0.0001% of the runtime, any amount of speed up is not worth making the code hard to read (from a person-hour perspective). Many times, it is difficult to gain more than a factor of 2 speed up in runtime. Definitely think about the potential gain in speed-up versus how long it takes to write and maintian the code.

## Some Ways to Speed up Your Code

If you identified the critical chunk of code to speed up, the general strategy is to remove unncessary computations as much as possible. There's unfortunately not one single method to speed up your code. But here are a few ideas:

1. Eliminate computations that are not being used (e.g., computing variables you do not use; processing parts of images that will be discarded). 
2. Use `numpy` functions whenever possible for matrix operations
3. Avoid using python `for` and `while` loops as they are slow
4. If MKL/BLAS is being run inside of already-parallelized code, disable MKL/BLAS.
5. Reduce the number of iterations or increase the tolerance of routines that are unncessarily precise (e.g., optimizers can run for less iterations; sin and cos can be approximated by taylor expansions). 
6. Avoid copying variables when they do not need to be copied (e.g., if the input is already a numpy.array, you don't need to wrap it with `np.array()` in ensure it's a numpy array). 
7. Turn some of your python code into C-code and call it with Python
8. Other ideas?



## Code Performance Extrapolation

If your code takes 24 hours to run to completion, or needs to be run on a cluster, it is likely still easier to develop and benchmark it on your laptop. We can use extrapolation to estimate performance. 

### Runtime

Generally algorithm runtimes scale as N^a where a is some positive number greater than 1, and N is the size of your data. (In CS terms, this is often disccused in "Big O" notation as  $O(N^a)$ runtime). If your data is too big or takes too long to benchmark on a single machine, try running your code on much smaller data. If you do this for ~3-5 different test data with different N, you can estimate what is the scaling of your code (i.e., what is "a"). Once you know the scaling you can use that scaling relation to extrapolate how long it will take on larger datasets. If you can also use more cores, you can divide the final runtime by the number of cores you can use. 

For example, my code takes 10 seconds on 10x10 data, 10,000 seconds on 100x100 data, and 40,000 seconds on 200x200 data (all on a single core). My code scales as N^2 where N is the size of one dimension of the data. If I want to run it on 10,000x10,000 data with 100 cores, my code should take (10,000^2)/100 = 1,000,000 seconds to run.

### Memory

If you are worried that your program will run out of memory, try calculating how much memory your program uses. On modern 64-bit machines, a single number (an integer, float) takes up 8 bytes. If you have an array of 1000x1000 floats, then it will take up 8e6 bytes or 8 MBs. If you have 3-D numerical data with dimensions 1000x1000x1000, then it will take up 8e9 bytes or 8 GBs.